In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import pandas as pd
import numpy as np
import datetime


import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", color_codes=True)


dateparse = lambda x: datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')

# Read data 
df=pd.read_csv("../input/911.csv",
    header=0,names=['lat', 'lng','desc','zip','title','timeStamp','twp','addr','e'],
    dtype={'lat':str,'lng':str,'desc':str,'zip':str,
                  'title':str,'timeStamp':str,'twp':str,'addr':str,'e':int}, 
     parse_dates=['timeStamp'],date_parser=dateparse)


# Set index
df.index = pd.DatetimeIndex(df.timeStamp)
df=df[(df.timeStamp >= "2016-01-01 00:00:00")]

In [ ]:
df.head()

In [ ]:
#Basic Info

In [ ]:
df.info()

In [ ]:
#The top 5 zip codes 

In [ ]:
df['zip'].value_counts().head()

In [ ]:
#Top 5 Townships

In [ ]:
df['twp'].value_counts().head(5)

In [ ]:
#Making Reason column, ex EMS: BACK PAINS/INJURY, Reason  = EMS

In [ ]:
def newCol(rsn):
    d = rsn.split(':')
    return d[0]

In [ ]:
df['Reason'] = list(map(newCol,df['title']))

In [ ]:
df['Reason'].value_counts()

In [ ]:
#EMS is the Reason most calls are made

In [ ]:
# Reason plot

In [ ]:
sns.countplot('Reason',data= df)

In [ ]:
#Splitting date time into Hour, Month, Week

In [ ]:
df['Hour'] = df['timeStamp'].apply(lambda t : t.hour)
df['Month'] = df['timeStamp'].apply(lambda t: t.month)
df['Week'] = df['timeStamp'].apply(lambda t:t.dayofweek)

In [ ]:
#new Data
df.head()

In [ ]:
#converting week to Monday - Sunday

In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
df['Day of the Week'] = df['Week'].map(dmap)

In [ ]:
#Dropping column e
df =df.drop('e',axis=1)

In [ ]:
#countplot of the Day of Week column with the hue Reason

In [ ]:
sns.countplot('Day of the Week',data=df,hue='Reason')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
#for month
sns.countplot('Month',data=df,hue='Reason')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
#grouping by month
byMonth = df.groupby('Month').count()
byMonth.head()

In [ ]:
#count of calls per month
byMonth['twp'].plot()

In [ ]:
# a linear fit to the plot
sns.lmplot(x= 'Month',y='twp',data=byMonth.reset_index())

In [ ]:
#new column date
df['Date'] = df['timeStamp'].apply(lambda t : t.date())
df['Date'].head()

In [ ]:
#grouping by date and count of calls
d = df.groupby('Date').count()['twp'].plot()

In [ ]:
#seperating plot based off of reason
df[df['Reason']=='Traffic'].groupby('Date').count()['twp'].plot()


In [ ]:
df[df['Reason']=='Fire'].groupby('Date').count()['twp'].plot(figsize=(7,4))

In [ ]:
df[df['Reason']=='EMS'].groupby('Date').count()['twp'].plot()

In [ ]:
#restructuring df , making hour the column
df1 = df.groupby(by = ['Day of the Week','Hour']).count()['Reason'].unstack()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df1,cmap='inferno')

In [ ]:
#Month as the column
df2 = df.groupby(by=['Day of the Week','Month']).count()['twp'].unstack()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df2,cmap='inferno')